In [118]:
import pandas as pd
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB, GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.impute import SimpleImputer

In [119]:
def read_sqlite(file):
    import sqlite3
    from pandas import read_sql_query, read_sql_table

    with sqlite3.connect(file) as dbcon:
        tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", dbcon)['name'])
        out = {tbl : read_sql_query(f"SELECT * from {tbl}", dbcon) for tbl in tables}

    return out

In [120]:
df = read_sqlite('data/database.sqlite')


In [121]:
df.keys()

dict_keys(['betfront', 'football_data'])

In [122]:
df['football_data']

,Season,Datetime,Div,Country,League,Referee,HomeTeam,AwayTeam,FTHG,FTAG,...,VCCD,VCCH,VCD,VCH,WHA,WHCA,WHCD,WHCH,WHD,WHH
0,2021,2021-05-31 23:00:00,ARG,Argentina,Primera Division,None,Independiente,Colon Santa FE,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,2021-05-31 19:00:00,ARG,Argentina,Primera Division,None,Racing Club,Boca Juniors,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,2021-05-31 02:30:00,USA,USA,MLS,None,Seattle Sounders,Austin FC,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020/2021,2021-05-31 02:15:00,MEX,Mexico,Liga MX,None,Cruz Azul,Santos Laguna,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,2021-05-31 00:30:00,BRA,Brazil,Serie A,None,Internacional,Sport Recife,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190668,2000/2001,2000-07-29 00:00:00,F2,France,Division 2,None,Ajaccio,Le Mans,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190669,2000/2001,2000-07-29 00:00:00,SC0,Scotland,Premier League,Willie Young,St Mirren,Kilmarnock,0,1,...,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.25,2.40
190670,2000/2001,2000-07-28 00:00:00,F1,France,Le Championnat,None,Paris SG,Strasbourg,3,1,...,NaN,NaN,NaN,NaN,6.5,NaN,NaN,NaN,3.70,1.40
190671,2000/2001,2000-07-28 00:00:00,F1,France,Le Championnat,None,Marseille,Troyes,3,1,...,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.50,1.45


In [123]:
total_cells = df['football_data'].shape[0] * df['football_data'].shape[1]
total_missing = df['football_data'].isna().sum().sum()
perc_missing = round((total_missing/total_cells) * 100, 2)

perc_missing

66.47

In [124]:
bets = df['betfront']

In [125]:
bets['COMPETION'].unique()

array(['world-cup-2010', 'premier-league-2009', 'tippeligaen-2009',
       'major-league-soccer-2009', 'allsvenskan-2009',
       'veikkausliiga-2009', 'campeonato-brasileiro-2009',
       'campeonato-brasileiro-b-2009', 'champions-league-2009-2010',
       'europa-league-2009-2010', 'super-league-2009-2010',
       'erste-liga-2009-2010', 'bundesliga-2009-2010',
       'superliga-2009-2010', 'gambrinus-liga-2009-2010',
       'ekstraklasa-2009-2010', 'jupiler-league-2009-2010',
       'eredivisie-2009-2010', 'a-league-2009-2010',
       'championship-2009-2010', 'ligue-1-2009-2010',
       '1st-division-2009-2010', 'portuguese-liga-2009-2010',
       'premier-league-2009-2010', 'primera-division-2009-2010',
       'serie-a-2009-2010', 'belgacom-league-2009-2010',
       'adeccoligaen-2009', 'division-1-2009', 'primera-division-2010',
       'premier-league-2010', 'j-league-2010', 'allsvenskan-2010',
       'tippeligaen-2010', 'major-league-soccer-2010',
       'veikkausliiga-2010', 'e

In [126]:
 bets['DATETIME']

0        2009-02-11 22:30:00
1        2009-02-11 23:30:00
2        2009-02-11 23:45:00
3        2009-02-12 04:15:00
4        2009-02-12 06:30:00
                ...         
37848    2013-02-25 03:00:00
37849    2013-02-25 03:00:00
37850    2013-02-26 23:45:00
37851    2013-02-26 23:45:00
37852    2013-02-27 00:00:00
Name: DATETIME, Length: 37853, dtype: object

In [127]:
bets['DATETIME'] = pd.to_datetime(bets['DATETIME'])

In [128]:
bets.loc[bets['DATETIME'].dt.year == 2015]

,YEAR,DATETIME,COUNTRY,COMPETION,MATCH,HOME_OPENING,DRAW_OPENING,AWAY_OPENING,HOME_CLOSING,DRAW_CLOSING,AWAY_CLOSING,HT,FT,TOTAL_FT,HTG1,HTG2,FTG1,FTG2


In [129]:
# merging the files
joined_files = os.path.join("data/EPL/", "EPL*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)

In [130]:
joined_list

['data/EPL/EPL2007-2008.csv',
 'data/EPL/EPL2003-2004.csv',
 'data/EPL/EPL2015-2016.csv',
 'data/EPL/EPL2009-2010.csv',
 'data/EPL/EPL2014-2015.csv',
 'data/EPL/EPL2004-2005.csv',
 'data/EPL/EPL2013-2014.csv',
 'data/EPL/EPL2005-2006.csv',
 'data/EPL/EPL2017-2018.csv',
 'data/EPL/EPL2012-2013.csv',
 'data/EPL/EPL2019-2020.csv',
 'data/EPL/EPL2016-2017.csv',
 'data/EPL/EPL2018-2019.csv',
 'data/EPL/EPL2008-2009.csv',
 'data/EPL/EPL2011-2012.csv',
 'data/EPL/EPL2010-2011.csv',
 'data/EPL/EPL2006-2007.csv',
 'data/EPL/EPL2020-2021.csv']

In [131]:
joined_list

['data/EPL/EPL2007-2008.csv',
 'data/EPL/EPL2003-2004.csv',
 'data/EPL/EPL2015-2016.csv',
 'data/EPL/EPL2009-2010.csv',
 'data/EPL/EPL2014-2015.csv',
 'data/EPL/EPL2004-2005.csv',
 'data/EPL/EPL2013-2014.csv',
 'data/EPL/EPL2005-2006.csv',
 'data/EPL/EPL2017-2018.csv',
 'data/EPL/EPL2012-2013.csv',
 'data/EPL/EPL2019-2020.csv',
 'data/EPL/EPL2016-2017.csv',
 'data/EPL/EPL2018-2019.csv',
 'data/EPL/EPL2008-2009.csv',
 'data/EPL/EPL2011-2012.csv',
 'data/EPL/EPL2010-2011.csv',
 'data/EPL/EPL2006-2007.csv',
 'data/EPL/EPL2020-2021.csv']

In [132]:
for x in range(len(joined_list)):
    print(joined_list[x])
    test = pd.read_csv(joined_list[x])
    
    print(test.shape)
    print(test.columns)
    print("*"*20)

data/EPL/EPL2007-2008.csv
(380, 71)
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'GBH', 'GBD', 'GBA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'SBH', 'SBD', 'SBA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA', 'BSH', 'BSD', 'BSA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD',
       'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA',
       'BbAvAHA'],
      dtype='object')
********************
data/EPL/EPL2003-2004.csv
(380, 57)
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'GBH', 'GBD',
       'GBA', 'IWH', 'I

In [133]:
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [134]:

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/07,Aston Villa,Liverpool,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,11/08/07,Bolton,Newcastle,1.0,3.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,11/08/07,Derby,Portsmouth,2.0,2.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,11/08/07,Everton,Wigan,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,11/08/07,Middlesbrough,Blackburn,1.0,2.0,A,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,E0,19/01/2021,West Ham,West Brom,2.0,1.0,H,1.0,0.0,H,...,2.08,-1.00,1.88,2.02,1.90,2.03,1.94,2.06,1.89,1.98
6641,E0,19/01/2021,Leicester,Chelsea,2.0,0.0,H,2.0,0.0,H,...,2.01,0.00,2.05,1.85,2.08,1.85,2.09,1.92,2.03,1.84
6642,E0,20/01/2021,Man City,Aston Villa,2.0,0.0,H,0.0,0.0,D,...,2.62,-1.50,1.98,1.95,1.97,1.95,2.00,2.07,1.92,1.95
6643,E0,20/01/2021,Fulham,Man United,1.0,2.0,A,1.0,1.0,D,...,2.15,1.00,1.75,2.05,1.79,2.15,1.90,2.16,1.80,2.09


In [135]:
df['Date'] = pd.to_datetime(df['Date'])

In [136]:
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2007-11-08,Aston Villa,Liverpool,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2007-11-08,Bolton,Newcastle,1.0,3.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2007-11-08,Derby,Portsmouth,2.0,2.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2007-11-08,Everton,Wigan,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2007-11-08,Middlesbrough,Blackburn,1.0,2.0,A,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,E0,2021-01-19,West Ham,West Brom,2.0,1.0,H,1.0,0.0,H,...,2.08,-1.00,1.88,2.02,1.90,2.03,1.94,2.06,1.89,1.98
6641,E0,2021-01-19,Leicester,Chelsea,2.0,0.0,H,2.0,0.0,H,...,2.01,0.00,2.05,1.85,2.08,1.85,2.09,1.92,2.03,1.84
6642,E0,2021-01-20,Man City,Aston Villa,2.0,0.0,H,0.0,0.0,D,...,2.62,-1.50,1.98,1.95,1.97,1.95,2.00,2.07,1.92,1.95
6643,E0,2021-01-20,Fulham,Man United,1.0,2.0,A,1.0,1.0,D,...,2.15,1.00,1.75,2.05,1.79,2.15,1.90,2.16,1.80,2.09


In [137]:
df = df.sort_values(by='Date')
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
480,E0,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,E0,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,E0,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,E0,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,E0,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,E0,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,...,2.78,1.25,1.77,2.02,1.87,2.07,1.92,2.13,1.84,2.04
6627,E0,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,...,1.53,0.00,1.82,2.11,1.82,2.13,1.84,2.19,1.77,2.12
6626,E0,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,...,2.26,1.25,2.02,1.91,2.00,1.93,2.04,1.94,1.98,1.90
6625,E0,2021-12-01,Sheffield United,Newcastle,1.0,0.0,H,0.0,0.0,D,...,1.59,-0.25,1.99,1.94,1.99,1.93,2.10,1.96,1.97,1.92


In [138]:
df = df[df['Div'] == 'E0']

In [139]:
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
480,E0,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,E0,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,E0,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,E0,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,E0,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,E0,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,...,1.97,1.00,1.90,2.00,1.92,2.01,1.96,2.08,1.86,2.01
6624,E0,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,...,2.78,1.25,1.77,2.02,1.87,2.07,1.92,2.13,1.84,2.04
6627,E0,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,...,1.53,0.00,1.82,2.11,1.82,2.13,1.84,2.19,1.77,2.12
6626,E0,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,...,2.26,1.25,2.02,1.91,2.00,1.93,2.04,1.94,1.98,1.90


In [140]:
df.isna().sum()

Div            0
Date           0
HomeTeam       0
AwayTeam       0
FTHG           0
            ... 
PCAHA       6080
MaxCAHH     6080
MaxCAHA     6080
AvgCAHH     6080
AvgCAHA     6080
Length: 151, dtype: int64

In [141]:
# Dropping irrevelant betting columns 
# Focusing mainly 365 bets
df.drop(df.iloc[:, 26:], inplace=True, axis=1)

/Users/bryan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [142]:
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
480,E0,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,...,11.0,3.0,1.0,0.0,0.0,0.0,0.0,4.50,3.40,1.72
486,E0,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,...,6.0,7.0,5.0,3.0,0.0,0.0,0.0,1.72,3.40,4.50
485,E0,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,...,16.0,4.0,6.0,1.0,1.0,0.0,0.0,2.20,3.40,2.87
484,E0,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,...,17.0,15.0,3.0,2.0,3.0,0.0,1.0,1.57,3.60,5.50
481,E0,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,...,11.0,4.0,3.0,1.0,0.0,0.0,0.0,5.50,3.60,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,E0,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,...,11.0,3.0,6.0,0.0,2.0,0.0,0.0,5.25,4.20,1.61
6624,E0,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,...,12.0,1.0,10.0,1.0,3.0,0.0,0.0,5.00,4.33,1.61
6627,E0,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,...,11.0,6.0,5.0,2.0,2.0,0.0,0.0,2.45,3.10,3.10
6626,E0,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,...,13.0,3.0,4.0,1.0,2.0,0.0,0.0,7.00,4.50,1.45


In [143]:
#Dropping irrevelant column 
df = df.drop(columns='Div')

In [144]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
480,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,J Winter,...,11.0,3.0,1.0,0.0,0.0,0.0,0.0,4.50,3.40,1.72
486,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,U Rennie,...,6.0,7.0,5.0,3.0,0.0,0.0,0.0,1.72,3.40,4.50
485,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,A Wiley,...,16.0,4.0,6.0,1.0,1.0,0.0,0.0,2.20,3.40,2.87
484,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,M Messias,...,17.0,15.0,3.0,2.0,3.0,0.0,1.0,1.57,3.60,5.50
481,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,M Dean,...,11.0,4.0,3.0,1.0,0.0,0.0,0.0,5.50,3.60,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,R Jones,...,11.0,3.0,6.0,0.0,2.0,0.0,0.0,5.25,4.20,1.61
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,A Marriner,...,12.0,1.0,10.0,1.0,3.0,0.0,0.0,5.00,4.33,1.61
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,M Atkinson,...,11.0,6.0,5.0,2.0,2.0,0.0,0.0,2.45,3.10,3.10
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,K Friend,...,13.0,3.0,4.0,1.0,2.0,0.0,0.0,7.00,4.50,1.45


In [145]:
# Renaming Columns for better understanding
df.columns = ['Date','HomeTeam','AwayTeam','FT_Home_Goal','FT_Away_Goal','FT_Result','HT_Home_Goal','HT_Away_Goal',
                        'HT_Result','Referee','H_Shots','A_Shots','H_Shots_Target','A_Shots_Target','H_Foul',
                        'A_Foul','H_Corner','A_Corner','H_Yellow','A_Yellow','H_Red','A_Red', 'HomeOdds', 'DrawOdds', 'AwayOdds']

In [146]:
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,Referee,...,A_Foul,H_Corner,A_Corner,H_Yellow,A_Yellow,H_Red,A_Red,HomeOdds,DrawOdds,AwayOdds
480,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,J Winter,...,11.0,3.0,1.0,0.0,0.0,0.0,0.0,4.50,3.40,1.72
486,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,U Rennie,...,6.0,7.0,5.0,3.0,0.0,0.0,0.0,1.72,3.40,4.50
485,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,A Wiley,...,16.0,4.0,6.0,1.0,1.0,0.0,0.0,2.20,3.40,2.87
484,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,M Messias,...,17.0,15.0,3.0,2.0,3.0,0.0,1.0,1.57,3.60,5.50
481,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,M Dean,...,11.0,4.0,3.0,1.0,0.0,0.0,0.0,5.50,3.60,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,R Jones,...,11.0,3.0,6.0,0.0,2.0,0.0,0.0,5.25,4.20,1.61
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,A Marriner,...,12.0,1.0,10.0,1.0,3.0,0.0,0.0,5.00,4.33,1.61
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,M Atkinson,...,11.0,6.0,5.0,2.0,2.0,0.0,0.0,2.45,3.10,3.10
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,K Friend,...,13.0,3.0,4.0,1.0,2.0,0.0,0.0,7.00,4.50,1.45


In [147]:
df.dtypes

Date              datetime64[ns]
HomeTeam                  object
AwayTeam                  object
FT_Home_Goal             float64
FT_Away_Goal             float64
FT_Result                 object
HT_Home_Goal             float64
HT_Away_Goal             float64
HT_Result                 object
Referee                   object
H_Shots                  float64
A_Shots                  float64
H_Shots_Target           float64
A_Shots_Target           float64
H_Foul                   float64
A_Foul                   float64
H_Corner                 float64
A_Corner                 float64
H_Yellow                 float64
A_Yellow                 float64
H_Red                    float64
A_Red                    float64
HomeOdds                 float64
DrawOdds                 float64
AwayOdds                 float64
dtype: object

In [148]:
 df.columns.values

array(['Date', 'HomeTeam', 'AwayTeam', 'FT_Home_Goal', 'FT_Away_Goal',
       'FT_Result', 'HT_Home_Goal', 'HT_Away_Goal', 'HT_Result',
       'Referee', 'H_Shots', 'A_Shots', 'H_Shots_Target',
       'A_Shots_Target', 'H_Foul', 'A_Foul', 'H_Corner', 'A_Corner',
       'H_Yellow', 'A_Yellow', 'H_Red', 'A_Red', 'HomeOdds', 'DrawOdds',
       'AwayOdds'], dtype=object)

In [149]:
# Dropped irrevelant data that cant forecast a game prior 
df.drop(['Referee', 'H_Foul', 'A_Foul', 'H_Corner', 'A_Corner',
         'H_Yellow', 'A_Yellow', 'H_Red', 'A_Red'], axis=1, inplace=True)

In [150]:
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,H_Shots,A_Shots,H_Shots_Target,A_Shots_Target,HomeOdds,DrawOdds,AwayOdds
480,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,12.0,17.0,4.0,7.0,4.50,3.40,1.72
486,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,10.0,25.0,6.0,15.0,1.72,3.40,4.50
485,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,8.0,13.0,5.0,9.0,2.20,3.40,2.87
484,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,22.0,11.0,13.0,6.0,1.57,3.60,5.50
481,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,9.0,13.0,5.0,9.0,5.50,3.60,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,8.0,9.0,2.0,3.0,5.25,4.20,1.61
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,7.0,17.0,3.0,1.0,5.00,4.33,1.61
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,12.0,5.0,3.0,3.0,2.45,3.10,3.10
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,11.0,13.0,0.0,7.0,7.00,4.50,1.45


In [151]:
# If the home team wins its true 
# If the away team wins its false 
# If they draw then its false 
def Winner(Result):
    if Result == 'H':
        return True
    else:
        return False
# create a new column based on condition
df['Winner'] = df['FT_Result'].apply(Winner)
# display the dataframe
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,H_Shots,A_Shots,H_Shots_Target,A_Shots_Target,HomeOdds,DrawOdds,AwayOdds,Winner
480,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,12.0,17.0,4.0,7.0,4.50,3.40,1.72,False
486,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,10.0,25.0,6.0,15.0,1.72,3.40,4.50,False
485,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,8.0,13.0,5.0,9.0,2.20,3.40,2.87,False
484,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,22.0,11.0,13.0,6.0,1.57,3.60,5.50,False
481,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,9.0,13.0,5.0,9.0,5.50,3.60,1.57,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,8.0,9.0,2.0,3.0,5.25,4.20,1.61,False
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,7.0,17.0,3.0,1.0,5.00,4.33,1.61,True
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,12.0,5.0,3.0,3.0,2.45,3.10,3.10,False
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,11.0,13.0,0.0,7.0,7.00,4.50,1.45,False


In [152]:
def HomeAdvantage(Result):
    if Result == 'H':
        return 1
    else:
        return 0
# create a new column based on condition
df['Home_Advantage'] = df['FT_Result'].apply(HomeAdvantage)
# display the dataframe
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,H_Shots,A_Shots,H_Shots_Target,A_Shots_Target,HomeOdds,DrawOdds,AwayOdds,Winner,Home_Advantage
480,2003-01-11,Everton,Chelsea,0.0,1.0,A,0.0,0.0,D,12.0,17.0,4.0,7.0,4.50,3.40,1.72,False,0
486,2003-01-11,Tottenham,Bolton,0.0,1.0,A,0.0,0.0,D,10.0,25.0,6.0,15.0,1.72,3.40,4.50,False,0
485,2003-01-11,Southampton,Man City,0.0,2.0,A,0.0,1.0,A,8.0,13.0,5.0,9.0,2.20,3.40,2.87,False,0
484,2003-01-11,Newcastle,Aston Villa,1.0,1.0,D,1.0,1.0,D,22.0,11.0,13.0,6.0,1.57,3.60,5.50,False,0
481,2003-01-11,Leeds,Arsenal,1.0,4.0,A,0.0,3.0,A,9.0,13.0,5.0,9.0,5.50,3.60,1.57,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,8.0,9.0,2.0,3.0,5.25,4.20,1.61,False,0
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,7.0,17.0,3.0,1.0,5.00,4.33,1.61,True,1
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,12.0,5.0,3.0,3.0,2.45,3.10,3.10,False,0
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,11.0,13.0,0.0,7.0,7.00,4.50,1.45,False,0


In [153]:
goal_diff = {}
goal_diff = ((df.groupby('HomeTeam')["FT_Home_Goal"].sum() - 
              df.groupby('AwayTeam')["FT_Away_Goal"].sum()).to_dict())
print (goal_diff)

{'Arsenal': 181.0, 'Aston Villa': 26.0, 'Birmingham': 42.0, 'Blackburn': 53.0, 'Blackpool': 5.0, 'Bolton': 52.0, 'Bournemouth': 31.0, 'Brighton': 16.0, 'Burnley': 25.0, 'Cardiff': 16.0, 'Charlton': 30.0, 'Chelsea': 155.0, 'Crystal Palace': -5.0, 'Derby': 4.0, 'Everton': 172.0, 'Fulham': 120.0, 'Huddersfield': 2.0, 'Hull': 33.0, 'Leeds': 6.0, 'Leicester': 5.0, 'Liverpool': 180.0, 'Man City': 195.0, 'Man United': 142.0, 'Middlesbrough': 61.0, 'Newcastle': 126.0, 'Norwich': 60.0, 'Portsmouth': 76.0, 'QPR': 5.0, 'Reading': 6.0, 'Sheffield United': 27.0, 'Southampton': 69.0, 'Stoke': 98.0, 'Sunderland': 46.0, 'Swansea': 52.0, 'Tottenham': 140.0, 'Watford': 37.0, 'West Brom': 75.0, 'West Ham': 108.0, 'Wigan': 22.0, 'Wolves': 29.0}


In [154]:
# Focusing on 2015-2020 seasons to be more accurate
df = df[df['Date'] >= '2015-08-08']

In [155]:
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,H_Shots,A_Shots,H_Shots_Target,A_Shots_Target,HomeOdds,DrawOdds,AwayOdds,Winner,Home_Advantage
760,2015-08-08,Bournemouth,Aston Villa,0.0,1.0,A,0.0,0.0,D,11.0,7.0,2.0,3.0,2.00,3.60,4.00,False,0
761,2015-08-08,Chelsea,Swansea,2.0,2.0,D,2.0,1.0,H,11.0,18.0,3.0,10.0,1.36,5.00,11.00,False,0
762,2015-08-08,Everton,Watford,2.0,2.0,D,0.0,1.0,A,10.0,11.0,5.0,5.0,1.70,3.90,5.50,False,0
763,2015-08-08,Leicester,Sunderland,4.0,2.0,H,3.0,0.0,H,19.0,10.0,8.0,5.0,1.95,3.50,4.33,True,1
764,2015-08-08,Man United,Tottenham,1.0,0.0,H,1.0,0.0,H,9.0,9.0,1.0,4.0,1.65,4.00,6.00,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,8.0,9.0,2.0,3.0,5.25,4.20,1.61,False,0
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,7.0,17.0,3.0,1.0,5.00,4.33,1.61,True,1
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,12.0,5.0,3.0,3.0,2.45,3.10,3.10,False,0
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,11.0,13.0,0.0,7.0,7.00,4.50,1.45,False,0


In [156]:
df.dtypes

Date              datetime64[ns]
HomeTeam                  object
AwayTeam                  object
FT_Home_Goal             float64
FT_Away_Goal             float64
FT_Result                 object
HT_Home_Goal             float64
HT_Away_Goal             float64
HT_Result                 object
H_Shots                  float64
A_Shots                  float64
H_Shots_Target           float64
A_Shots_Target           float64
HomeOdds                 float64
DrawOdds                 float64
AwayOdds                 float64
Winner                      bool
Home_Advantage             int64
dtype: object

In [157]:
df['Home_Advantage'] = df['Home_Advantage'].astype(float)

<ipython-input-157-438139c2eb6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Home_Advantage'] = df['Home_Advantage'].astype(float)


In [158]:
df

,Date,HomeTeam,AwayTeam,FT_Home_Goal,FT_Away_Goal,FT_Result,HT_Home_Goal,HT_Away_Goal,HT_Result,H_Shots,A_Shots,H_Shots_Target,A_Shots_Target,HomeOdds,DrawOdds,AwayOdds,Winner,Home_Advantage
760,2015-08-08,Bournemouth,Aston Villa,0.0,1.0,A,0.0,0.0,D,11.0,7.0,2.0,3.0,2.00,3.60,4.00,False,0.0
761,2015-08-08,Chelsea,Swansea,2.0,2.0,D,2.0,1.0,H,11.0,18.0,3.0,10.0,1.36,5.00,11.00,False,0.0
762,2015-08-08,Everton,Watford,2.0,2.0,D,0.0,1.0,A,10.0,11.0,5.0,5.0,1.70,3.90,5.50,False,0.0
763,2015-08-08,Leicester,Sunderland,4.0,2.0,H,3.0,0.0,H,19.0,10.0,8.0,5.0,1.95,3.50,4.33,True,1.0
764,2015-08-08,Man United,Tottenham,1.0,0.0,H,1.0,0.0,H,9.0,9.0,1.0,4.0,1.65,4.00,6.00,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,2021-03-01,Newcastle,Leicester,1.0,2.0,A,0.0,0.0,D,8.0,9.0,2.0,3.0,5.25,4.20,1.61,False,0.0
6624,2021-04-01,Southampton,Liverpool,1.0,0.0,H,1.0,0.0,H,7.0,17.0,3.0,1.0,5.00,4.33,1.61,True,1.0
6627,2021-12-01,Wolves,Everton,1.0,2.0,A,1.0,1.0,D,12.0,5.0,3.0,3.0,2.45,3.10,3.10,False,0.0
6626,2021-12-01,Burnley,Man United,0.0,1.0,A,0.0,0.0,D,11.0,13.0,0.0,7.0,7.00,4.50,1.45,False,0.0
